## Demonstration of usage of the Hidden Markov Model code

The hidden_markov_model class requires NumPy, but only NumPy. The other packages we use are for data loading and preprocessing.

Import required libraries.

In [1]:
import numpy as np
import pandas as pd

The problem we are trying to solve is to infer from a given set of observation of die throws, which die are fair and which are loaded. We will use the Viterbi algorithm to solve this problem. 

Initialize our initial guesses for the initial probability of each state at a timestep t, the transition probabilities, and the emission probabilities. We initialize our states to 0 and 1 because there are two possible states at each time step.

In [10]:
init_prob = np.array([.5, .5])
transition_prob = np.array(([0.95, .05], [.05, .95]))
emission_prob = np.array(([1/6, 1/6, 1/6, 1/6, 1/6, 1/6], [1/10, 1/10, 1/10, 1/10, 1/10, 1/2]))
states = np.array([0,1])

Reading in the data.

In [26]:
data1 = pd.read_csv('~/Documents/machine-learning/data/hw11pb1.csv', header=None)
data2 = pd.read_csv('~/Documents/machine-learning/data/hw11pb2.csv', header=None)

Convert our Pandas DataFrames to NumPy arrays and flatten them.

In [27]:
data1 = data1.values.flatten()
data2 = data2.values.flatten()

Creating the Hidden Markov Model class. 

It takes as arguments some starting probabilities, transition probabilities, emission probabilities, the set of possible states, and the observation set.

In [28]:
class hidden_markov_model:
    def __init__(self, start_probs, transition_probs, emission_probs, state_set, observations):
        self.start = start_probs
        self.transition = transition_probs
        self.emission = emission_probs
        self.states = state_set
        self.obs = observations
        self.n_states = len(self.states)
        self.t_obs = len(observations)
        self.alpha = np.zeros((self.t_obs, self.n_states))
        self.beta = np.zeros((self.t_obs, self.n_states))
        self.underflow = np.ones(len(self.obs))
    
    def get_gammas(self, alphas, betas, T, N):
        gammas = np.zeros((len(self.obs)-1, self.n_states))
        xs = np.zeros((len(self.obs)-1, self.n_states, self.n_states))
        for t in range(self.t_obs-1):
            xinum = np.array([self.alpha[t]]*2).T * self.transition * np.array([self.beta[t+1]]*2)*np.array([self.emission[:, self.obs[t]-1]]*2)
            xs[t] = xinum/np.sum(xinum)
            gammas[t] = np.sum(xs[t], axis=1)
        
        return gammas, xs
        
    def EM(self, N, M, T, pi, A, B):
        alphas = self.forward()
        betas = self.backward()
        
        gammas, xs = self.get_gammas(alphas, betas, T, N)
        
        new_pi = gammas[0, :]
        
        new_a = np.sum(xs, axis=0)/(np.array([np.sum(gammas, axis=0)]*2).T)
       
        xinds = np.zeros((len(self.obs), len(self.emission[0])))
        xinds[np.arange(len(self.obs)), self.obs-1] = 1
        
        new_b = (gammas.T@xinds[:-1])/np.array([np.sum(gammas, axis=0)]*6).T
        
        return new_pi, new_a, new_b
        
    def baumwelch(self, tolerance, maxiter):
        T = self.t_obs
        N = self.n_states
        M = len(self.states)
        likely = []
        
        pi = np.log(self.start)
        A = np.log(self.transition)
        B = np.log(self.emission)
        
        iter_ = 0
        err = np.inf
        while iter_ < maxiter or err > tolerance:
            iter_ += 1
            if iter_ % 100 == 0:
                print('Iter:', iter_)
            new_pi, new_a, new_b = self.EM(N, M, T, pi, A, B)
            
            err = np.abs(np.linalg.norm(A - new_a) + np.linalg.norm(B - new_b))/2
            
            pi, A, B = new_pi, new_a, new_b
            
        return pi, A, B
    
    def forward(self):
        self.alpha[0] = self.start * self.emission[:, self.obs[0]-1]
        for t in range(1, self.t_obs):
            for k in self.states:
                self.alpha[t, k] = (self.alpha[t-1]@self.transition[k]) * self.emission[k, self.obs[t]-1]
            self.underflow[t] = np.sum(self.alpha[t])
            self.alpha[t] = self.alpha[t]/self.underflow[t]
        return self.alpha
    
    def backward(self):
        self.beta[:, -1:] = 1
        
        for t in range(2, len(self.obs)+1):
            for k in self.states:
                self.beta[-t, k] = np.sum(self.beta[-t+1] * self.transition[k] * self.emission[:, self.obs[-t+1]-1])
            self.beta[-t] = self.beta[-t]/self.underflow[-t]
        return self.beta
    
    def viterbi(self):
        N = self.transition.shape[0]
        delta = np.zeros((self.t_obs, N))
        psi = np.zeros((self.t_obs, N))
        delta[0] = self.start * self.emission[:, self.obs[0]-1]
        for tt in range(1, self.t_obs):
            for j in range(N):
                delta[tt, j] = np.max(delta[tt-1] * self.transition[:, j] * self.emission[j, self.obs[tt]-1])
                psi[tt, j] = np.argmax(delta[tt-1] * self.transition[:, j])
                
        states = np.zeros(self.t_obs)
        states[self.t_obs-1] = np.argmax(np.exp(delta[self.n_states-1]))
        states = states.astype('int')
        for t in range(self.t_obs-2, -1, -1):
            states[t] = psi[t+1, states[t+1]]
        return states
    
    

Creating an instance of the Hidden Markov Model class with the probability estimates created above.

In [15]:
hmm1 = hidden_markov_model(init_prob, transition_prob, emission_prob, states, data1)

Here using the Viterbi algorithm to solve for which die are fair and which are loaded.

In [16]:
guess = hmm1.viterbi()
empt = np.empty(guess.shape, dtype='str')
f_inds = np.where(guess == 0)
l_inds = np.where(guess==1)
empt[f_inds] = 'F'
empt[l_inds] = 'L'
empt

array(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'L', 'L', 'L',
       'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L',
       'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L',
       'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L',
       'L', 'L', 'L', 'L', 'L', 'L', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F',
       'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F

Here, we are solving a second problem using a second dataset. This dataset contains 1000 dice rolls, so we have much more information about the state of the die than before. We are going to use this increased information to solve for the model parameters using the Baum-Welch algorithm.

In cell below, initializing the model with a guess of the parameters.

In [21]:
A = np.array([[0.95, 0.05], [0.05, 0.95]])
B = np.array(([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6], [.1, .1, .1, .1, .1, .5]]))
pi = np.array([1/2, 1/2])

Creating a hidden_markov_model object with the initial probability guesses above. 

In [22]:
hmm2 = hidden_markov_model(pi, A, B, states, data2)

Solving for the true model parameters using Baum-Welch

In [23]:
solved_pi, solved_A, solve_B = hmm2.baumwelch(1e-2, 100)

Iter: 100


In [24]:
print('Pi found through Baum Welch:', solved_pi)
print('Transition probabilities found through Baum Welch:', solved_A)
print('Emission probabilities found through Baum Welch:', solve_B)

Pi found through Baum Welch: [0.42671712 0.57328288]
Transition probabilities found through Baum Welch: [[0.99065118 0.00934882]
 [0.16993597 0.83006403]]
Emission probabilities found through Baum Welch: [[0.20145444 0.20746921 0.19321639 0.2003796  0.1272622  0.07021816]
 [0.10984433 0.10203983 0.1023266  0.10441057 0.06631893 0.51505975]]
